The friedman test

In [4]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import requests
import json
from pandas.io.json import json_normalize

import utils as utils

import sys
from tqdm import tqdm
%pylab inline
plt.style.use("bmh")
np.random.seed(983456)

Populating the interactive namespace from numpy and matplotlib


In [5]:
es = Elasticsearch([{'host': '35.178.209.118', 'use_ssl': False, 'read timeout':20}]) 

In [20]:
def fetch_res_pmids(name,es,index):
    include=["pmid", "last_author_name","last_author.forename","last_author.affiliation"]
    es_query={"query":{"term":{"last_author_name": name }},"_source":{"includes":include},"size":10000} 
    res=es.search(index=index,
                 body=es_query)
    # res=json_normalize(res)
    # return res
    #print(es_query)
    #print(res)
    return pd.DataFrame([h["_source"] for h in res["hits"]["hits"]])

In [21]:
df=fetch_res_pmids('Friedman, N',es,"bulk_papers5")

In [22]:
df.pmid

0     25249765
1     26606974
2     25171404
3     25024161
4     24244020
5     26446933
6     12740579
7     17190847
8     29514105
9     23219532
10    23334501
11    24421210
12    24312094
13    23935451
14    23352245
15    23769506
16    22925037
17    11560351
18    12151303
19    12161879
20    12176835
21    11344753
22    15454407
23    15262806
24    12843144
25    14764868
26    12015876
27    11473012
28    21444810
29    21109421
30    20529911
31    21685068
32    21685097
33    19968953
34    19846608
35    18463706
36    16117504
37    16597232
38    18586706
39    18494234
40    17237082
41    10154065
42    28731407
43    28716917
44    25801168
45    27496019
46    22984176
47    23089682
48    23072772
49    29903938
50    30605682
51    31464369
Name: pmid, dtype: object

In [23]:
def fetch_res_detail(pmid,es,index):
    include=["pmid", "doi","last_author_id","last_author_name","last_author_forename","last_author.affiliation","last_author_email","title","pub_year","last_author_country","last_author_inst",\
    "journal_name","language","authors","mesh","mesh_major"]
    es_query={"query":{"terms":{"pmid": pmid }},"_source":{"includes":include},"size":10000} 
    res=es.search(index=index,
                 body=es_query)
    # res=json_normalize(res)
    # return res
    #print(es_query)
    #print(res)
    return pd.DataFrame([h["_source"] for h in res["hits"]["hits"]])

In [ ]:
fetch_res_detail(pmid,es,index)